In [1]:
library("btergm")
library("dplyr")
library("Matrix")
library("statnet")  
library("texreg") 
library("data.table")
library("metafor")

Loading required package: xergm.common
Loading required package: ergm
Loading required package: network
network: Classes for Relational Data
Version 1.16.0 created on 2019-11-30.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


ergm: version 3.10.4, created on 2019-06-10
Copyright (c) 2019, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvine
                    Steven M. Goodreau, University of Washington
                    Pavel N. Krivitsky, University of Wollongong
     

Data Preparation #TODO Split into separate cells

In [2]:
#Data Preparation 
create_round_edges <- function(data, players){
  players_in_game <- union(unique(data$player_id), unique(data$other_id))
  edge_mat <- matrix(0,nrow=players,ncol=players)
  data <- adjust_missing_players(data, players_in_game)
  for(j in 1:nrow(data)){
    player_id <- data[[j, "player_id"]]
    other_id <- data[[j, "other_id"]]
    edge_mat[player_id, other_id] <- 1
  }
  return(edge_mat)
}

add_endowment <- function(network, data){
  endowment <- unique(data[(data$round==1), ])
  endowment <- endowment[ , c("player_id", "endowment")][order(endowment$player_id), ]
  network <- set.vertex.attribute(network, "endowment", endowment[,2])
  return(network)
}

add_wealth <- function(network, data, round){
  wealth_data <- as.data.table(data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  wealth_data <- transform(wealth_data, wealth=sqrt(wealth))
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  wealth_data <- adjust_missing_players(wealth_data, players_in_game)
  setDF(wealth_data)
  round_wealth <- get_round_wealth(wealth_data, round)
  network <- set.vertex.attribute(network, "wealth", round_wealth)
  return(network)
}

adjust_missing_players <- function(data, players_in_game){
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    data$player_id[data$player_id > x] <- data$player_id[data$player_id > x] - 1
    if("other_id" %in% colnames(data))
    {
      data$other_id[data$other_id > x] <- data$other_id[data$other_id > x] - 1
    }
  }
  return(data)
}

get_round_wealth <- function(data, current_round){
  round_data <- data[(data$round==current_round), ]
  return(round_data[,"wealth"])
}

get_xergm_data <- function(data1, data2){
  network <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], num_players)
    network[[i]] <- network(round_edges)
    network[[i]] <- add_endowment(network[[i]], data2)
    network[[i]] <- add_wealth(network[[i]], data2, i)
    odegsqrt <- degree(network[[i]], cmode = "outdegree")
    network[[i]] <- set.vertex.attribute(network[[i]],
                                         "odegsqrt", odegsqrt)
  }
  return(network)
}

Load Data #TODO

In [4]:
interact_data <- read.csv("interactions.csv", header=T)
wealth_data <- read.csv("finwealth_from_interactions.csv", header=T)

Run Models #TODO

In [5]:
all_networks <- list()
all_models <- list()
for (t in unique(interact_data$treatment)) {
  group_networks <- list()
  group_models <- list()
  for (g in unique(interact_data[(interact_data$treatment==t), ]$group_id)) {
    filtered_data1 <- interact_data[(interact_data$treatment==t) & (interact_data$group_id==g), ]
    filtered_data2 <- wealth_data[(wealth_data$treatment==t) & (wealth_data$group_id==g), ]
    xergm_network <- get_xergm_data(filtered_data1, filtered_data2)
    xergm_model <- btergm(xergm_network ~ edges+ mutual + ttriple +
                           + nodeocov("odegsqrt") +
                           nodeicov("endowment") + nodeocov("endowment") + nodematch("endowment")
                         + nodeicov("wealth") + nodeocov("wealth") + nodematch("wealth"), R = 5)
    group_networks[[paste(t, '_', g, sep="")]] <- xergm_network
    group_models[[paste(t, '_', g, sep="")]] <- xergm_model
    print(paste(t, '_', g, sep=""))

  }
  all_models[[paste(t, sep="")]] <- group_models
  all_networks[[paste(t, sep="")]] <- group_networks
}


No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 5 bootstrapping replications on a single computing core...
Warning message:
“`set_attrs()` is deprecated as of rlang 0.3.0
This warning is displayed once per session.”Done.


[1] "Endow_6"



No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 5 bootstrapping replications on a single computing core...
Done.


[1] "Endow_14"



No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 5 bootstrapping replications on a single computing core...
Done.


...

[1] "Wealth + Endow_41"



No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 5 bootstrapping replications on a single computing core...
Done.


[1] "Wealth + Endow_42"



No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 5 bootstrapping replications on a single computing core...
Done.


[1] "Wealth + Endow_43"


In [6]:
yis <-matrix(list(), nrow=10, ncol=20)
seis <- matrix(list(), nrow=10, ncol=20)
terms <- 0
for(i in 1:length(all_models[[1]])){
  model <- all_models[[1]][[i]]
  stats <- btergm.se(model)
  terms = stats
  for(j in 1:nrow(stats)){
    yis[i,j] = stats[j,1]
    seis[i,j] = stats[j,2]
  }
}
print(rownames(terms))
yis <- split(yis, rep(1:ncol(yis), each = nrow(yis)))
seis <- split(seis, rep(1:ncol(seis), each = nrow(seis)))
coefs <- list()
#Fix terms
for(i in 1:nrow(terms)){
  yi <- yis[i]
  sei <- seis[i]
  yi<- unlist(yi, use.names=FALSE)
  sei<- unlist(sei, use.names=FALSE)
  coef <- rownames(terms)[i]
  new.term <- data.frame(yi, sei)
  meta<- rma.uni(yi, sei, data = new.term, method = "REML", level=95)
  coefs[[coef]] <- meta
}

print(coefs[["mutual"]])

Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of boots

 [1] "edges"               "mutual"              "ttriple"            
 [4] "nodeocov.odegsqrt"   "nodeicov.endowment"  "nodeocov.endowment" 
 [7] "nodematch.endowment" "nodeicov.wealth"     "nodeocov.wealth"    
[10] "nodematch.wealth"   

Random-Effects Model (k = 10; tau^2 estimator: REML)

tau^2 (estimated amount of total heterogeneity): 0 (SE = 0.0502)
tau (square root of estimated tau^2 value):      0
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity: 
Q(df = 9) = 3.8932, p-val = 0.9183

Model Results:

estimate      se     zval    pval   ci.lb   ci.ub     
  1.2201  0.1058  11.5284  <.0001  1.0127  1.4275  ***

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1 

